# Generalizations of Max-Flow

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    e, v = dict(),dict()
    
    def read_graph(infile):
        with open(infile, 'r') as f:
            for line in f.readlines():
                line = line.strip().split(' ')
                if line[0] == 'p':
                    number = int(line[2])
                if line[0] == 'n':
                    v[int(line[1])] = int(line[2])
                if line[0] == 'a':
                    edge = (int(line[1]), int(line[2]))
                    if edge not in e.keys():
                        value = dict()
                        value['capacity'] = [int(line[4])]
                        value['cost'] = [float(line[5])]
                        e[edge] = value
                    else:
                        e[edge]['capacity'].append(int(line[4]))
                        e[edge]['cost'].append(float(line[5]))
        for edge in e.keys():
            if len(e[edge]['capacity']) > 1:
                for k in range(len(e[edge]['capacity'])):
                    number += 1
                    edge1 = (edge[0],number)
                    value = dict()
                    value['capacity'] = e[edge]['capacity'][k]
                    value['cost'] = e[edge]['cost'][k]/2.
                    e[edge1] = value
                    edge2 = (number, edge[1])
                    value = dict()
                    value['capacity'] = e[edge]['capacity'][k]
                    value['cost'] = e[edge]['cost'][k]/2.
                    e[edge2] = value
                del e[edge]
            
    def final_graph(e,v):    
        G = nx.DiGraph()
        for edge in e.keys():
            G.add_edge(edge[0], edge[1])
        for node1 in v:
            G.node[node1]['demand'] = v[node1]
        for (s1, s2) in G.edges():
            G.edge[s1][s2]['capacity'] = e[(s1, s2)]['capacity']
            G.edge[s1][s2]['weight'] = e[(s1, s2)]['cost']
        return G
    
    read_graph(infile)
    return final_graph(e,v)
    

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

In [3]:
from pulp import *

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    solver, weight = LpProblem("min_cost_flow", LpMinimize),dict()
    
    
    for edge in G.edges():
        weight[str(edge)] = G.get_edge_data(edge[0], edge[1])['weight']
    flow = pulp.LpVariable.dict("flow", [str(edge) for edge in G.edges()], 0)
    
    
    
    for edge in G.edges():
        solver += flow[str(edge)] <= G.get_edge_data(edge[0], edge[1])['capacity']
        
        
        
    for node in G.nodes():
        if G.node[node] != {}:
            demand = G.node[node]['demand']
        else:
            demand = 0
        f_in ,f_out = [],[]
        for edge in G.edges():
            if node == edge[0]:
                f_out.append(flow[str(edge)])   
            if node == edge[1]:
                f_in.append(flow[str(edge)])
            
        solver += lpSum(f_in) - lpSum(f_out) == demand
        
        
    solver += lpSum([weight[str((i,j))]*flow[str((i,j))] for (i,j) in G.edges()])
    status = solver.solve()    
    return int(value(solver.objective))   

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
